We want to use the model and algorithm of Tutorial--Van der Pol's dynamics with spike train observations.ipynb on the data in the folder Data.

In [17]:
# Libraries
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
import sys

# .py
sys.path.append("..")
from inference import GaussMarkovLagrange
from likelihoods import PointProcess
from mappings import AffineMapping
from transition import FixedPointSparseGP, SparseGP
from kernels import RBF
from linkfunctions import Exp
from models import PointProcessGPSDEmodel, GPSDE
sys.path.append("../Data")
from Load_plot_data import load_neuron_data, roster_plot

# Load the data and modify its format

In [22]:
#Load data
ids,times=load_neuron_data('../Data/Cellline1_Date190328_Chip2135.npz')
N=np.max(ids) # Number of neurons
trLen=np.array([np.max(times)]) #Tmax must be an array to have one for each trial

The data from the files is of the form
( id : neurons ids,
times : times of spike )

For the model we need a list of array where array i contains the times at which neuron i had a spike

We can observe using unique and some tests that the ids are all the numbers between 0 and 1016 except 131 and 899. We suppose that they are not present beacause they didn't have a spike.

In [23]:
Yspike=[]
for i in range(N):
    Yspike.append(times[ids==i])
len(Yspike)

#For now there is only one trial, to change if we want to define trials
Yspike=[Yspike]

# Choices for the model

In [24]:
#Choice of variables
dtgrid = 0.004 # discretisation for solving ODEs
xDim = 2 # two latents

In [25]:
#Choice of model components
link = Exp() # exponential link function (to define nonlinearity in likelihood )
like = PointProcess(Yspike, link, trLen, dtstep=dtgrid, nLeg=100) # point process likelihood 
kern = RBF(xDim) # RBF kernel

#To initialize the Affine mapping from latent to observations
C = 2.*np.random.rand(xDim,N) * np.random.choice([-1,1],size=(xDim,N)) 
d = 0.1*np.random.randn(1,N)
outputMapping = AffineMapping(torch.tensor(C), torch.tensor(d)) # affine output mapping, used inn the forward of the model

In [26]:
#Choose the inducing points and transition function

# generater inducing point locations on a 2D grid (for sparse gp)
xmin, xmax = -2., 2.
Zs1, Zs2 = torch.meshgrid([torch.linspace(xmin,xmax,5), torch.linspace(xmin,xmax,5)])
numZ = 25
Zs = torch.cat((Zs1.reshape(-1,1),Zs2.reshape(-1,1)),dim=1)

transfunc = SparseGP(kern, Zs) # choose sparse GP as a transition function
transfunc.q_mu.data = torch.randn(transfunc.q_mu.size()).type(torch.float64) # random initialisation for inducing point posterior mean

# uncomment to instead condition on fixed points
#Zs_fx = torch.tensor([0., 0.]).view(-1,xDim)
# transition function conditioned on fixed point
#transfunc = SparseGP(kern, Zs, Zs_fx) 

d:\logiciel\miniconda\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


# Build the model

In [27]:
#Build the model

# build point procces generative model for continuous time spike-time observations
model = PointProcessGPSDEmodel(xDim, transfunc, outputMapping, like, nLeg=100)

# assemble inference algorithm
inference = GaussMarkovLagrange(xDim, trLen, learningRate=1, dtstep=dtgrid)

# create GPSDE model object (final object)
myGPSDE = GPSDE(model, inference)

# fix inducing points on chosen grid
myGPSDE.model.transfunc.Zs.requires_grad = False